In [1]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
from scipy.stats import norm, multivariate_normal
import itertools
from itertools import combinations_with_replacement, combinations, permutations
import copy
import Tasmanian
import matplotlib.pyplot as plt

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pickle
import gzip

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy import integrate

import warnings
warnings.simplefilter("ignore", category=RuntimeWarning)
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-31 17:18:40.862213: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 17:18:40.862269: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 17:18:40.864104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-31 17:18:40.872957: I tensorflow/core/platform/cpu_feature_guard

In [52]:
#Create Test Functions
def func1(x1, x2):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    return 1 - x1**2 - x2**2

def func2(x1, x2):
    return np.maximum(func1(x1,x2), 0)

def func3(x1, x2):
    return 0.5*(np.sqrt(func1(x1, x2)**2 + 1e-5) + func1(x1, x2))

def func4(x1, x2):
    return np.sin(x1)*x2

def func5(x1, x2):
    return x1*x2

#Create test functions*pdf(normal)

def get_mean_covar(vars, mean = False, covar = False):
    np.random.seed = 1
    if mean == True:
        mean = np.array([1.0,1.2])#np.ones(vars.shape[1])
    else:
        mean = np.zeros(vars.shape[1])
    if covar == True:
        B = np.ones((vars.shape[1], vars.shape[1]))*0.5
        covar = np.dot(B.T, B) + 0.5*np.eye(vars.shape[1])
    else:
        covar = np.eye(vars.shape[1])

    return mean, covar

def func1_pdf(x1, x2, mean = True, covar = True):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
        
    np.random.seed = 1
    mean, covar = get_mean_covar(vars, mean = mean, covar = covar)
    pdf = multivariate_normal.pdf(vars, mean=mean, cov=covar)

    return func1(x1,x2)*pdf

def func2_pdf(x1, x2, mean = True, covar = True):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
        
    np.random.seed = 1
    mean, covar = get_mean_covar(vars, mean = mean, covar = covar)
    pdf = multivariate_normal.pdf(vars, mean=mean, cov=covar)

    return func2(x1,x2)*pdf

def func3_pdf(x1, x2, mean = True, covar = True):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
        
    np.random.seed = 1
    mean, covar = get_mean_covar(vars, mean = mean, covar = covar)
    pdf = multivariate_normal.pdf(vars, mean=mean, cov=covar)

    return func3(x1,x2)*pdf

def func4_pdf(x1, x2, mean = True, covar = True):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
        
    np.random.seed = 1
    mean, covar = get_mean_covar(vars, mean = mean, covar = covar)
    pdf = multivariate_normal.pdf(vars, mean=mean, cov=covar)

    return func4(x1,x2)*pdf

def func5_pdf(x1, x2, mean = True, covar = True):
    # for 2D: f(x)= 1 - x1^2 - x2^2
    try:
        vars = np.concatenate((x1,x2),axis =1)
    except:
        vars = np.array([x1,x2]).reshape(1,-1)
        
    np.random.seed = 1
    mean, covar = get_mean_covar(vars, mean = mean, covar = covar)
    pdf = multivariate_normal.pdf(vars, mean=mean, cov=covar)

    return func5(x1,x2)*pdf

In [53]:
#Test MC vs Sparse Grid on a fxn*norm.pdf w/ gaussian random variable
def sg_integ_gauss(func, dim, gp_mean, gp_covar, depth = 20, seed = 1):
    np.random.seed = seed
    #Get grid points and weights
    grid_p = Tasmanian.SparseGrid()
    grid_p.makeGlobalGrid(dim, 0, depth, "hyperbolic", 'gauss-hermite')
    points_p_old = grid_p.getPoints()
    weights_p = grid_p.getQuadratureWeights()
            
    #Diagonalize covariance matrix
    L = scipy.linalg.cholesky(np.real(gp_covar), lower=True)
    transformed_points = L@points_p_old.T
    points_p = gp_mean[:, np.newaxis].T + np.sqrt(2)*(transformed_points.T)

    #Evaluate function
    if dim == 2:
        y = func(points_p[:,0], points_p[:,1])
    elif dim ==5:
        y = func(points_p[:,0], points_p[:,1], points_p[:,2],points_p[:,3],points_p[:,4])

    # print(points_p.shape)
    # print(weights_p.shape)
    # print(y.shape)

    integ = (np.pi**(-dim/2))*y@weights_p.T

    return integ

def mc_integ_gauss(func, dim, n, gp_mean, gp_covar, seed = 1):
    # Monte Carlo integration of given function over domain from a to b (for each parameter)
    # dim: dimensions of function
    #Initialize total ei  
    # n = math.ceil((norm.ppf(1-0.05/2)/0.05)**2)  
    # np.random.seed(seed)

    #Get random variable from normal distribution
    x_list = np.random.multivariate_normal(gp_mean, gp_covar, n)

    #Evaluate function
    if dim == 2:
        y = func(x_list[:,0], x_list[:,1])
    elif dim ==5:
        y = func(x_list[:,0], x_list[:,1], x_list[:,2],x_list[:,3],x_list[:,4])

    y_mean =  np.mean(y)

    integ = y_mean

    return integ


In [54]:
#Test MC vs Sparse Grid on a fxn*norm.pdf w/ gaussian random variable
column_names = ["scipy", "mc n=200", "mc n=2000", "mc n=20000", "sg depth=5", "sg depth=10", "sg depth=20"]
# column_names = ["scipy","mc n=100", "sg depth=5"]
results_df = pd.DataFrame(columns=column_names)
func_list = [func1, func2, func3, func4, func5] #, func5, func6, func7, func8, func9]
func_list2 = [func1_pdf,  func2_pdf, func3_pdf, func4_pdf, func5_pdf]
# func_list2 = [func_pdf, func2_pdf, func3_pdf, func4_pdf, func5_pdf, func6_pdf, func7_pdf, func8_pdf, func9_pdf] 
# func_list = [func1, func2, func3, func4, func5, func6, func7, func8, func9]

l_bound = -1
u_bound = 1
seed = 1
depth1 = 5
depth2 = 10
depth3 = 20
n1 = 200
n2 = 2000
n3 = 20000
add_mean = True
add_var = True

for i in range(len(func_list)):
    func = func_list[i]
    func_pdf = func_list2[i]

    if i > 5:
        #Scipy intractable for 5D problems
        z_scipy = "n/a"

        dim =5
    else:
        z_scipy = integrate.dblquad(func_pdf, -np.inf, np.inf, -np.inf, np.inf, args = (add_mean, add_var))[0]
        dim =2

    dummy_data = np.ones(dim).reshape(1,-1)
    mean, covar = get_mean_covar(dummy_data, mean=add_mean, covar = add_var)
    print("Mean:", mean)
    print("Covar:", covar)
    #Evaluate "improvement (the normal function)" integral using MC
    z_mc1 = mc_integ_gauss(func, dim, n1, mean, covar)
    z_mc2 = mc_integ_gauss(func, dim, n2, mean, covar)
    z_mc3 = mc_integ_gauss(func, dim, n3, mean, covar)
    #Evaluate "improvement (the normal function)" integral using sparse grid
    z_sg1 = sg_integ_gauss(func, dim, mean, covar, depth1)
    z_sg2 = sg_integ_gauss(func, dim, mean, covar, depth2)
    z_sg3 = sg_integ_gauss(func, dim, mean, covar, depth3)

    #Add to dataframe results
    iter_df = pd.DataFrame(columns=column_names)
    integ_results = [z_scipy, z_mc1, z_mc2, z_mc3, z_sg1, z_sg2, z_sg3]
    # integ_results = [z_scipy, z_mc1, z_sg1]
    # Add the new row to the DataFrame
    iter_df.loc[0] = integ_results
    results_df = pd.concat([results_df.astype(iter_df.dtypes), iter_df], ignore_index=True)

Mean: [1.  1.2]
Covar: [[1.  0.5]
 [0.5 1. ]]
Mean: [1.  1.2]
Covar: [[1.  0.5]
 [0.5 1. ]]
Mean: [1.  1.2]
Covar: [[1.  0.5]
 [0.5 1. ]]
Mean: [1.  1.2]
Covar: [[1.  0.5]
 [0.5 1. ]]
Mean: [1.  1.2]
Covar: [[1.  0.5]
 [0.5 1. ]]


In [55]:
results_df

,scipy,mc n=200,mc n=2000,mc n=20000,sg depth=5,sg depth=10,sg depth=20
0,-3.440000,-3.156743,-3.608276,-3.421413,-3.440000,-3.440000,-3.440000
1,0.112555,0.128471,0.116949,0.113582,0.121543,0.073127,0.033579
2,0.112562,0.089453,0.114757,0.111974,0.121818,0.072913,0.033344
3,0.776308,0.760078,0.782342,0.777003,0.776268,0.776308,0.776308
4,1.700000,1.843702,1.745191,1.684011,1.700000,1.700000,1.700000


In [ ]:
results_df.to_csv("mc_test_gauss_all_hyper.csv")